In [ ]:
# Imports
!pip install -q transformers[torch] datasets pysentimiento accelerate evaluate emoji
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding, pipeline
import random
import torch
import torch.nn.functional as F
import re
import emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 23.4 MB/s eta 0:00:00


In [ ]:
# Eliminar saltos de línea y espacios repetidos
def delete_spaces(comment):
    spaces_pattern = r'[\n\r]+|\s+'
    return re.sub(spaces_pattern, ' ', comment)

# Convertir todo a minúsculas
def lower_text(comment):
    return comment.lower()

# Eliminar URL
def delete_urls(comment):
    url_pattern = r'http[s]?://\S+'
    return re.sub(url_pattern, '', comment)

# Eliminar consonantes repetidas y puntos suspensivos
def delete_repeated_consonants(comment):
    repeated_consonant_pattern = r'([^aeiou\s\r\n0-9])\1{1,}'
    def replace(match):
        char = match.group(1)
        if char in 'rcnl':
            return char * 2
        else:
            return char

    return re.sub(repeated_consonant_pattern, replace, comment, flags=re.IGNORECASE)

# Dejar como máximo 2 vocales iguales contiguas
def delete_repeated_vowels(comment):
    repeated_vowels_pattern = r'([aeiouAEIOU])\1{2,}'
    return re.sub(repeated_vowels_pattern, r'\1\1', comment, flags=re.IGNORECASE)

# Eliminar acentos no empleados en Español
def delete_accents (comment):
    comment = re.sub(r"[àâãäå]", "a", comment)
    comment = re.sub(r"ç", "c", comment)
    comment = re.sub(r"[èêë]", "e", comment)
    comment = re.sub(r"[ìîï]", "i", comment)
    comment = re.sub(r"[òôõö]", "o", comment)
    comment = re.sub(r"[ùû]", "u", comment)
    comment = re.sub(r"[ýÿ]", "y", comment)
    return comment

# Eliminar caracteres inusuales
def delete_characters(comment):
    special_characters = r'[ºª|·~¬\^`[\]¨´#\\\'\(\)*\<>_]'
    return re.sub(special_characters, '', comment)

# Eliminar otros caracteres inusuales
def delete_specific_characters(comment):
    specific_chars_pattern = r'[«»✈ºø♀♂€🇮±°ª‘’“🇱]'
    return re.sub(specific_chars_pattern, '', comment)

# Eliminar emoticonos
def delete_emoticons(comment):
    return emoji.replace_emoji(comment, replace='')

# Eliminar caracteres árabes
def delete_arabic_letters(text):
    arabic_pattern = r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]'
    return re.sub(arabic_pattern, '', text)

# Unificar las distintas formas de expresar la risa
def unify_laughs (comment):
    laugh_pattern = r"\b(a*ha+h[ha]*|o?l+o+l+[ol]*|x+d+[x*d*]*|a*ja+[j+a+]+|j+e+j+[ej]*|j+s+j+[sj]*|j+i+j+[ij]*)\b"
    return re.sub(laugh_pattern, 'jaja', comment, flags=re.IGNORECASE)

def delete_extra (comment):
  return re.sub(r'\bmikeldi\b', '', comment, flags=re.IGNORECASE)

# Función para preprocesar el texto
def preprocess_comment(comment):
    comment = delete_spaces(comment)
    comment = lower_text(comment)
    comment = delete_urls(comment)
    comment = delete_repeated_consonants(comment)
    comment = delete_repeated_vowels(comment)
    comment = delete_accents(comment)
    comment = delete_characters(comment)
    comment = delete_specific_characters(comment)
    comment = delete_emoticons(comment)
    comment = delete_arabic_letters(comment)
    comment = unify_laughs(comment)
    comment = delete_extra(comment)
    return comment

In [ ]:
def compute_metrics(pred):
    y_true = pred.label_ids
    y_pred = pred.predictions.argmax(-1)
    acc = accuracy_score(y_true, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
# Cargar el dataset
database = "amaiaruvi/news_racist_comments_spanish"
dataset = load_dataset(database)
dataset

Generating train split:   0%|          | 0/3005 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/438 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/851 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 3005
    })
    validation: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 438
    })
    test: Dataset({
        features: ['link', 'title', 'comment', 'racist'],
        num_rows: 851
    })
})

In [ ]:
# Cargar el modelo
modelo = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(modelo)
# BertForSequenceClassification ya incluye una capa de clasificación para tareas como clasificación de texto. Solo necesitas especificar el número de clases (en este caso, 2).
model = AutoModelForSequenceClassification.from_pretrained(modelo, num_labels=2)

tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.config

BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002
}

In [ ]:
tokenizer.special_tokens_map

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
# Este modelo admite 512 tokens por cada sentencia
tokenizer.model_max_length

512

In [ ]:
tokenizer.get_vocab()

{'incondicional': 21339,
 'infer': 4885,
 '##rie': 6529,
 'verdes': 11001,
 'fotógrafo': 17709,
 'estableciendo': 17801,
 '8': 997,
 '##rec': 1477,
 'pelear': 7743,
 '##rael': 3808,
 '[unused714]': 720,
 '##dor': 2436,
 'persistentes': 23720,
 'betty': 14421,
 'cama': 4256,
 'animales': 4701,
 'botellas': 17092,
 '##ippi': 28221,
 'benefician': 25445,
 'rí': 20306,
 'brasil': 5017,
 'queréis': 13458,
 '##pers': 22114,
 'sellado': 28244,
 'daga': 26024,
 'garcía': 7873,
 'alternativos': 21634,
 'destac': 4999,
 'reconocer': 6911,
 'primarios': 24200,
 '##telos': 30302,
 'guay': 14471,
 'joa': 12377,
 '[unused200]': 206,
 'multi': 4284,
 '##tc': 27567,
 'reporte': 12788,
 'antártida': 23461,
 'abri': 7485,
 'suble': 17981,
 'cuidar': 8665,
 'añadió': 13952,
 'desigu': 10444,
 'opresión': 24472,
 '##tar': 1148,
 'envié': 16254,
 'destruida': 17395,
 'bienvenidos': 9375,
 '##mentos': 3005,
 '##ington': 5077,
 'dinas': 12671,
 '##dge': 21240,
 'implicado': 24253,
 'alejados': 24715,
 'apúre

In [ ]:
print("Preprocessing data...")
preprocessed_data = dataset.map(lambda ex: {
    "comment": preprocess_comment(ex["comment"]),
    "title": preprocess_comment(ex["title"]),
    "label": ex["racist"]
})

Preprocessing data...


Map:   0%|          | 0/3005 [00:00<?, ? examples/s]

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

Map:   0%|          | 0/851 [00:00<?, ? examples/s]

In [ ]:
def custom_tokenizer(examples):
    return tokenizer(
        examples["comment"],
        examples["title"],
        padding=True,
        truncation=True,
        max_length=tokenizer.model_max_length,
    )

In [ ]:
preprocessed_data['test'][1]

{'link': 'https://okdiario.com/espana/vox-empapela-gerona-carteles-arabe-estas-espana-hombres-mujeres-tienen-mismos-derechos-12797483',
 'title': 'vox empapela gerona con carteles en árabe: «estás en españa, hombres y mujeres tienen los mismos derechos»',
 'comment': 'mira quien habla, los de los tiros en la nuca.',
 'racist': 0,
 'label': 0}

In [ ]:
tokenized = custom_tokenizer(preprocessed_data['test'][1])
tokens_strings = tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

print("Texto a tokenizar:", preprocessed_data['test'][1]['comment'], ' + ', preprocessed_data['test'][1]['title'])
print("Tokens:", tokens_strings)
print("\n\ninput_ids:", tokenized['input_ids'])
print("token_type_ids:", tokenized['token_type_ids'])
print("attention_mask:", tokenized['attention_mask'])

Texto a tokenizar: mira quien habla, los de los tiros en la nuca.  +  vox empapela gerona con carteles en árabe: «estás en españa, hombres y mujeres tienen los mismos derechos»
Tokens: ['[CLS]', 'mira', 'quien', 'habla', ',', 'los', 'de', 'los', 'tiros', 'en', 'la', 'nuca', '.', '[SEP]', 'vo', '##x', 'empa', '##pel', '##a', 'ger', '##ona', 'con', 'carteles', 'en', 'árabe', ':', '[UNK]', 'estás', 'en', 'españa', ',', 'hombres', 'y', 'mujeres', 'tienen', 'los', 'mismos', 'derechos', '[UNK]', '[SEP]']


input_ids: [4, 2065, 1925, 2892, 1019, 1067, 1009, 1067, 15737, 1035, 1032, 27509, 1008, 5, 2107, 30991, 7431, 13722, 30956, 4601, 1791, 1048, 19949, 1035, 6338, 995, 3, 1499, 1035, 2942, 1019, 2305, 1040, 2209, 2018, 1067, 4549, 1889, 3, 5]
token_type_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
print("Tokenizing data...")
encoded_data = preprocessed_data.map(custom_tokenizer, batched=True)
encoded_data = encoded_data.remove_columns(['link', 'title', 'comment', 'racist'])

Tokenizing data...


Map:   0%|          | 0/3005 [00:00<?, ? examples/s]

Map:   0%|          | 0/438 [00:00<?, ? examples/s]

Map:   0%|          | 0/851 [00:00<?, ? examples/s]

In [ ]:
# Hiperparámetros iniciales
epochs = 5
batch_size = 16
learning_rate = 2e-5
weight_decay = 0.01

warmup_proportion = 0.1
total_steps = (epochs * len(dataset['train'])) / batch_size
warmup_steps = int(warmup_proportion * total_steps)

training_args = TrainingArguments(
    output_dir='./results',
    logging_dir='./logs',
    evaluation_strategy="epoch",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps,
    eval_accumulation_steps=1,
    logging_steps=500,
    save_steps=1000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    train_dataset=encoded_data['train'],
    eval_dataset=encoded_data['validation'],
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Entrenamiento
print("Training the model...")
trainer.train()

Training the model...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Evaluar el modelo con el conjunto de validación: se queda con el mejor valor
print("Evaluating with validation set.")
trainer.evaluate()

In [ ]:
# Predicciones
print("Predictions:")
test_predictions = trainer.predict(encoded_data["test"])
y_true = test_predictions.label_ids

logits = test_predictions.predictions
# Convertir los logits a un tensor de PyTorch
logits_tensor = torch.tensor(logits)
# Aplicar la función softmax a los logits para obtener probabilidades
probabilities = F.softmax(logits_tensor, dim=1)
# Obtener las clases predichas (índice de la probabilidad más alta)
y_pred = torch.argmax(probabilities, dim=1)
reporte = classification_report(y_true, y_pred, output_dict=False)
print(reporte)

In [ ]:
# Evaluación del modelo
# Crear el `Trainer` con el conjunto de evaluación y la función para métricas
print("Test evaluating with trainer:")
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=encoded_data["test"],  # Conjunto de evaluación
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Función para calcular métricas
)

# Evaluar el modelo
trainer.evaluate()